In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import logging
import json
logging.basicConfig(filename='Extract_Dec17_Nov18_RewShoppers.log', level=logging.INFO)
samplerows=None

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [3]:
historical_daily_subclass_files=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data_subclasslevel/"))
df_historical_daily_subclass_files=pd.DataFrame({"file_path":historical_daily_subclass_files})
df_historical_daily_subclass_files['Week_Date']=df_historical_daily_subclass_files['file_path'].apply(lambda x: x.split("es_week_ending_")[1][:10])
df_historical_daily_subclass_files=df_historical_daily_subclass_files[df_historical_daily_subclass_files['Week_Date']>="2017-12-03"]
df_historical_daily_subclass_files=df_historical_daily_subclass_files.sort_values("Week_Date",ascending=True)

In [4]:
_daily_2018_subclass_files=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
_daily_2018_subclass_files=[x for x in _daily_2018_subclass_files if "aily" in x]
_daily_2018_subclass_files=[x for x in _daily_2018_subclass_files if ".txt" in x]
df_daily_2018_subclass_files=pd.DataFrame({"file_path":_daily_2018_subclass_files})

df_daily_2018_subclass_files['Week_Date']=df_daily_2018_subclass_files['file_path'].apply(lambda x: x.split("2018_by_weeks/MediaStorm_")[1][:10])
df_daily_2018_subclass_files=df_daily_2018_subclass_files[df_daily_2018_subclass_files['Week_Date']>=df_historical_daily_subclass_files['Week_Date'].max()]
df_daily_2018_subclass_files=df_daily_2018_subclass_files[df_daily_2018_subclass_files['Week_Date']<="2018-12-01"]

df_daily_2018_subclass_files=df_daily_2018_subclass_files.sort_values("Week_Date",ascending=True)


In [5]:
all_12_month_sales=df_historical_daily_subclass_files.append(df_daily_2018_subclass_files).reset_index()
del all_12_month_sales['index']
logging.info(str(all_12_month_sales.shape))
logging.info(all_12_month_sales['Week_Date'].min())
logging.info(all_12_month_sales['Week_Date'].max())

logging.info("Started: "+str(datetime.datetime.now()))

In [18]:
all_12_month_sales['file_path'][11]

'/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-02-24.txt'

In [19]:
test=pd.read_table('/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-02-24.txt',nrows=samplerows,dtype=str,sep="|")
test['subclass_transaction_amt']=test['subclass_transaction_amt'].astype(float)
test.head(2)

,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,subclass_transaction_units,subclass_transaction_amt
0,5270,2018-02-21,8588,774d4d4a0aeda8120b1914e93ff1bdcf919abc1e60e8fa...,11003,8,2,1.2
1,1939,2018-02-19,3222,cc7fd8a2c5e7f8f50ec706d2b30823660ee98820990181...,13002,1,1,3.0


In [23]:
test['subclass_transaction_amt'].sum()

112050489.7999998

In [25]:
all_12_month_sales.to_csv("/home/jian/test.csv")

In [9]:
all_shopped_rewards_Dec17_Nov18=pd.DataFrame()

for i in range(len(all_12_month_sales)):
    week_end_date=all_12_month_sales['Week_Date'][i]
    file=all_12_month_sales['file_path'][i]
    df=pd.read_table(file,nrows=samplerows,dtype=str,sep="|",usecols=["customer_id_hashed","subclass_transaction_amt"])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df=df[df['subclass_transaction_amt']>0]
    del df['subclass_transaction_amt']
    df=df.drop_duplicates()
    df['week_end']=week_end_date
    
    all_shopped_rewards_Dec17_Nov18=all_shopped_rewards_Dec17_Nov18.append(df)
    print(str(i)+"|"+str(datetime.datetime.now())+"|"+week_end_date)
all_shopped_rewards_list=all_shopped_rewards_Dec17_Nov18['customer_id_hashed'].unique().tolist()


0|2019-03-05 11:39:26.115268|2017-12-09
1|2019-03-05 11:42:02.096263|2017-12-16
2|2019-03-05 11:43:23.883880|2017-12-23
3|2019-03-05 11:43:47.135987|2017-12-30
4|2019-03-05 11:44:08.624007|2018-01-06
5|2019-03-05 11:44:32.664567|2018-01-13
6|2019-03-05 11:45:04.053614|2018-01-20
7|2019-03-05 11:45:42.354270|2018-01-27
8|2019-03-05 11:47:44.284181|2018-02-03
9|2019-03-05 11:48:10.646829|2018-02-10
10|2019-03-05 11:48:37.036333|2018-02-17
11|2019-03-05 11:49:01.335206|2018-02-24
12|2019-03-05 11:49:56.932930|2018-03-03
13|2019-03-05 11:50:39.991610|2018-03-10
14|2019-03-05 11:51:05.579779|2018-03-17
15|2019-03-05 11:51:28.636289|2018-03-24
16|2019-03-05 11:52:18.313553|2018-03-31
17|2019-03-05 11:52:56.938215|2018-04-07
18|2019-03-05 11:53:26.120130|2018-04-14
19|2019-03-05 11:53:48.401725|2018-04-21
20|2019-03-05 11:54:11.467436|2018-04-28
21|2019-03-05 11:54:46.377685|2018-05-05
22|2019-03-05 11:55:08.542151|2018-05-12
23|2019-03-05 11:55:28.428907|2018-05-19
24|2019-03-05 11:55:51.605

In [13]:
len(all_shopped_rewards_Dec17_Nov18['week_end'].unique())

52

In [86]:
with open("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/RewardsShoppersList_in_12_month_JL_"+str(datetime.datetime.now().date())+".json", 'w') as outfile:
    json.dump(all_shopped_rewards_list, outfile)

In [82]:
all_shopped_rewards_Dec17_Nov18.to_csv("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/RewardsShoppers_in_12_month_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [20]:
len(all_shopped_rewards_list)

16635546

In [21]:
all_shopped_rewards_Dec17_Nov18.head(2)

,customer_id_hashed,week_end
2,51b0bcc411caf31a9b457aef532d60543d0be24f3820bd...,2017-12-09
4,ae4351cc7373b3780fc64587cba52d82b535009df4e688...,2017-12-09


In [22]:
all_shopped_rewards_Dec17_Nov18.shape

(57933793, 2)